<a href="https://colab.research.google.com/github/Nidhizala1426/Text_based_emotion_detection_python/blob/main/text_emotion_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary Libraries

In [35]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

Import the Dataset

In [36]:
text_data = pd.read_csv('/content/drive/MyDrive/text_emotion.csv')

Cleanup Data by removing Unecessary Columns

In [37]:
text_data = text_data.drop(text_data[text_data.sentiment == 'anger'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'hate'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'surprise'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'fun'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'boredom'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'empty'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'relief'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'worry'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'enthusiasm'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'love'].index)
text_data = text_data.drop(text_data[text_data.sentiment == 'neutral'].index)

In [38]:
text_data.head()

,tweet_id,sentiment,author,content
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


PreProcessing Data

In [39]:
import nltk  
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet
      Package wordnet is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

Making Letters Lowercase

In [40]:
text_data['content'] = text_data['content'].apply(lambda p: " ".join(p.lower() for p in p.split()))

Stopword Removal

In [41]:
stopwords1 = stopwords.words('english')
text_data['content'] = text_data['content'].apply(lambda p: " ".join(p for p in p.split() if p not in stopwords1))

Removing Punctuations

In [42]:
text_data['content'] = text_data['content'].str.replace('[^\w\s]',' ')

Finding top rarest words

In [43]:
frequency = pd.Series(' '.join(text_data['content']).split()).value_counts()[-10000:]
frequency = list(frequency.index)
text_data['content'] = text_data['content'].apply(lambda p: " ".join(p for p in p.split() if p not in frequency))

Label encoding

In [44]:
#Labels sad=1 & happy=0
labels = preprocessing.LabelEncoder()
y = labels.fit_transform(text_data.sentiment.values)

Splitting the data

In [45]:
X_train, X_val, y_train, y_val = train_test_split(text_data.content.values, y, stratify=y, random_state=42, test_size=0.1, shuffle=True)

###TFIDF 

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
X_train_vect_tfidf = tfidf_vect.fit_transform(X_train)
X_val_vect_tfidf = tfidf_vect.fit_transform(X_val)

##Count Vectors

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
vect_count = CountVectorizer(analyzer='word')
vect_count.fit(text_data['content'])
count_X_train =  vect_count.transform(X_train)
count_X_val =  vect_count.transform(X_val)

#Models

##Naive Bayes TFIDF

In [48]:
naive = MultinomialNB()
naive.fit(X_train_vect_tfidf, y_train)
y_pred = naive.predict(X_val_vect_tfidf)
print('naive bayes tfidf accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred,labels=[0,1]))

naive bayes tfidf accuracy 0.5481695568400771
              precision    recall  f1-score   support

           0       0.55      0.54      0.54       521
           1       0.55      0.56      0.55       517

    accuracy                           0.55      1038
   macro avg       0.55      0.55      0.55      1038
weighted avg       0.55      0.55      0.55      1038



##Naive Bayes Count vector

In [49]:
naive_c = MultinomialNB()
naive_c.fit(count_X_train, y_train)
y_pred_c = naive_c.predict(count_X_val)
print('naive bayes count vectors accuracy %s' % accuracy_score(y_pred_c, y_val))
print(classification_report(y_val, y_pred_c,labels=[0,1]))

naive bayes count vectors accuracy 0.7716763005780347
              precision    recall  f1-score   support

           0       0.78      0.77      0.77       521
           1       0.77      0.78      0.77       517

    accuracy                           0.77      1038
   macro avg       0.77      0.77      0.77      1038
weighted avg       0.77      0.77      0.77      1038



##Linear SVM TFIDF

In [52]:
linear_svm_tf = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
linear_svm_tf.fit(X_train_vect_tfidf, y_train)
y_pred_svm = linear_svm_tf.predict(X_val_vect_tfidf)
print('svm using tfidf accuracy %s' % accuracy_score(y_pred_svm, y_val))
print(classification_report(y_val, y_pred_svm,labels=[0,1]))

svm using tfidf accuracy 0.5317919075144508
              precision    recall  f1-score   support

           0       0.54      0.43      0.48       521
           1       0.52      0.64      0.58       517

    accuracy                           0.53      1038
   macro avg       0.53      0.53      0.53      1038
weighted avg       0.53      0.53      0.53      1038



##Linear SVM Count Vector

In [51]:
linear_svm = SGDClassifier(alpha=0.001, random_state=5, max_iter=15, tol=None)
linear_svm.fit(count_X_train, y_train)
y_pred_sc = linear_svm.predict(count_X_val)
print('lsvm using count vectors accuracy %s' % accuracy_score(y_pred_sc, y_val))
print(classification_report(y_val, y_pred_sc,labels=[0,1]))

lsvm using count vectors accuracy 0.7928709055876686
              precision    recall  f1-score   support

           0       0.78      0.81      0.80       521
           1       0.80      0.77      0.79       517

    accuracy                           0.79      1038
   macro avg       0.79      0.79      0.79      1038
weighted avg       0.79      0.79      0.79      1038



##Logistic Regression TFIDF

In [53]:
logistic_reg = LogisticRegression(C=1)
logistic_reg.fit(X_train_vect_tfidf, y_train)
y_pred_LR1 = logistic_reg.predict(X_val_vect_tfidf)
print('log reg tfidf accuracy %s' % accuracy_score(y_pred_LR1, y_val))
print(classification_report(y_val, y_pred_LR1,labels=[0,1]))

log reg tfidf accuracy 0.5260115606936416
              precision    recall  f1-score   support

           0       0.53      0.47      0.50       521
           1       0.52      0.58      0.55       517

    accuracy                           0.53      1038
   macro avg       0.53      0.53      0.52      1038
weighted avg       0.53      0.53      0.52      1038



##Logistic Regression Count Vector

In [55]:
logistic_reg_c = LogisticRegression(C=1)
logistic_reg_c.fit(count_X_train, y_train)
y_pred_lr = logistic_reg_c.predict(count_X_val)
print('log reg count vectors accuracy %s' % accuracy_score(y_pred_lr, y_val))
print(classification_report(y_val, y_pred_lr,labels=[0,1]))

log reg count vectors accuracy 0.7861271676300579
              precision    recall  f1-score   support

           0       0.78      0.81      0.79       521
           1       0.80      0.76      0.78       517

    accuracy                           0.79      1038
   macro avg       0.79      0.79      0.79      1038
weighted avg       0.79      0.79      0.79      1038



##Random forest TFIDF

In [54]:
random_f = RandomForestClassifier(n_estimators=500)
random_f.fit(X_train_vect_tfidf, y_train)
y_pred_rf = random_f.predict(X_val_vect_tfidf)
print('random forest tfidf accuracy %s' % accuracy_score(y_pred_rf, y_val))
print(classification_report(y_val, y_pred_rf,labels=[0,1]))

random forest tfidf accuracy 0.5269749518304432
              precision    recall  f1-score   support

           0       0.53      0.54      0.53       521
           1       0.53      0.51      0.52       517

    accuracy                           0.53      1038
   macro avg       0.53      0.53      0.53      1038
weighted avg       0.53      0.53      0.53      1038



##Random Forest Count vector

In [58]:
rand_f = RandomForestClassifier(n_estimators=500)
rand_f.fit(count_X_train, y_train)
y_pred_rf2 = rand_f.predict(count_X_val)
print('random forest with count vectors accuracy %s' % accuracy_score(y_pred_rf2, y_val))
print(classification_report(y_val, y_pred_rf2,labels=[0,1]))

random forest with count vectors accuracy 0.75626204238921
              precision    recall  f1-score   support

           0       0.75      0.77      0.76       521
           1       0.76      0.74      0.75       517

    accuracy                           0.76      1038
   macro avg       0.76      0.76      0.76      1038
weighted avg       0.76      0.76      0.76      1038



##Single Statement Test

In [65]:
from textblob import Word
tweet1 = pd.DataFrame(['Today is a good day', 'It breaks my heart'])
tweet1[0] = tweet1[0].str.replace('[^\w\s]',' ')
stopwords2 = stopwords.words('english')
tweet1[0] = tweet1[0].apply(lambda a: " ".join(a for a in a.split() if a not in stopwords2))
tweet1[0] = tweet1[0].apply(lambda b: " ".join([Word(word).lemmatize() for word in b.split()]))
tweets_counting = vect_count.transform(tweets[0])
#Predicting the emotion of the tweet using our already trained linear SVM
tweets_prediction = linear_svm.predict(tweets_counting)
print("Simple prediction sentence")
print(tweets_prediction)

Simple prediction sentence
[0 1]


**References :**

X. (2020, May 23). Applied Machine Learning: Part 3 - The Research Nest. Medium. https://medium.com/the-research-nest/applied-machine-learning-part-3-3fd405842a18